## Usage of frameworks



Among the frameworks available for building LLM Applications, LangChain is one of the popular options. Since it's first release in October 2022 till now, it has seen around 44M downloads. Let's see why it has such high adoption rate.

LangChain provides a number of features that make it easy to develop LLM applications. Following are a few 
- A generic interface for all LLMs, which makes it easy to switch between different LLMs.
- A prompt management system that helps developers create and manage prompts for LLMs.
- A number of common utilities for working with LLMs, such as text summarization and question answering.

In this article, we'll see how the generic interface makes it easier to switch between different LLMs

Let us consider a simple use case to demonstrate this. In this use-case, we will generate a recipe with 1 ingredient (input from the user) as the main ingredient.

For this demonstration, we will use 2 LLMs - Llama-2-70b-chat model hosted on Azure Cloud and a model from HuggingFace Hub for inference. The Llama-2-70b-chat model is deployed in serverless mode on Azure cloud.

### Initialise

Load the environment variables
- URL of the LLM deployed on Azure and the corresponding API Key
- LLM model from the Hugging Face Hub

In [1]:
from dotenv import dotenv_values
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
config = dotenv_values(".env") 

AZURE_LLM_URL = config["AZURE_LLM_URL"]
AZURE_API_KEY = config["AZURE_API_KEY"]

HUGGINGFACEHUB_API_TOKEN = config["HUGGINGFACEHUB_API_TOKEN"]
HUGGINGFACEHUB_LLM = "huggingfaceh4/zephyr-7b-alpha"

TEMPERATURE = 0.8
MAX_TOKENS = 400

Defining a function to return a query for generating  a recipe with the given item as the main ingredient. 

In [2]:
def get_query_for_recipe(main_ingredient):
    QUERY = f"Generate a recipe with {main_ingredient} as the main ingredient"
    print(f"Query: {QUERY}")
    return QUERY

### Invoke LLM without using LangChain

#### Use a LLM Hosted on Azure 

Send a HTTP request with the query in a specific format to the LLM hosted on Azure cloud. And extract the response.

In [3]:
import urllib.request
import json

HEADERS = {'Content-Type':'application/json', 'Authorization': (f'Bearer {AZURE_API_KEY}')}

def askLLM(data):
    body = str.encode(json.dumps(data))
    req = urllib.request.Request(AZURE_LLM_URL, body, HEADERS)
    result = ""

    try:
        response = urllib.request.urlopen(req)

        result = response.read().decode("utf8", 'ignore')
        print("LLM - Inference:", result)
        result = json.loads(result)
        result = result['choices'][0]['message']['content']
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))
        
    return result

In [4]:
data =  {
  "messages": [
    {
      "role": "user",
      "content": get_query_for_recipe("rice")
    }
  ],
  "temperature": 0.8,
  "max_tokens": 500
}

answer = askLLM(data) # Taskes ~40 sec
print(answer)

Query: Generate a recipe with rice as the main ingredient
LLM - Inference: {"choices":[{"finish_reason":"length","index":0,"message":{"content":"  Sure! Here's a simple recipe for a delicious rice dish that's perfect for a weeknight dinner:\n\nSpicy Shrimp and Rice Bowl\n\nIngredients:\n\n* 1 cup uncooked white or brown rice\n* 2 cups water\n* 1 tablespoon olive oil\n* 1 small onion, diced\n* 2 cloves garlic, minced\n* 1 pound large shrimp, peeled and deveined\n* 1 teaspoon grated ginger\n* 1/2 teaspoon ground cumin\n* 1/2 teaspoon smoked paprika\n* 1/4 teaspoon cayenne pepper\n* Salt and pepper, to taste\n* Chopped fresh cilantro, for garnish\n* Lime wedges, for serving\n\nInstructions:\n\n1. Start by cooking the rice according to package instructions. In a medium saucepan, bring the rice and water to a boil over high heat. Once boiling, reduce the heat to low, cover the saucepan with a tight-fitting lid, and cook for 18-20 minutes or until the water is absorbed and the rice is cooked

In [5]:
data =  {
  "messages": [
    {
      "role": "user",
      "content": get_query_for_recipe("lentils")
    }
  ],
  "temperature": 0.8,
  "max_tokens": 500
}

answer = askLLM(data) # Taskes ~40 sec
print(answer)

Query: Generate a recipe with lentils as the main ingredient
LLM - Inference: {"choices":[{"finish_reason":"length","index":0,"message":{"content":"  Sure, here's a recipe for a delicious and healthy lentil dish:\n\nLentil and Vegetable Curry\n\nIngredients:\n\n* 1 cup brown or green lentils, rinsed and drained\n* 2 medium onions, chopped\n* 3 cloves garlic, minced\n* 2 medium carrots, peeled and chopped\n* 2 medium potatoes, peeled and chopped\n* 1 medium zucchini, chopped\n* 1 red bell pepper, chopped\n* 1 can diced tomatoes\n* 2 teaspoons curry powder\n* 1 teaspoon ground cumin\n* 1 teaspoon ground coriander\n* 1/2 teaspoon turmeric\n* 1/2 teaspoon cayenne pepper (optional)\n* 1 can coconut milk\n* 2 cups vegetable broth\n* Salt and pepper, to taste\n* Fresh cilantro, chopped (for garnish)\n\nInstructions:\n\n1. In a large pot or Dutch oven, heat 1 tablespoon of oil over medium heat.\n2. Add the onions, garlic, carrots, potatoes, zucchini, and red bell pepper. Cook for 5-7 minutes, 

#### Use LLM from HuggingFace Hub

For using the LLM hosted on HuggingFace Hub, the AutoTokenizer, AutoModelForCausalLM classes from the transformers module

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np

c:\Users\Vibhor\Documents\Sree\code\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Note: Using a simpler model as the other ones cannot be run locally
HUGGINGFACEHUB_LLM = "EleutherAI/gpt-neo-125M"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACEHUB_LLM, token=HUGGINGFACEHUB_API_TOKEN)
# Load the model - Took ~ 1:30 min
model = AutoModelForCausalLM.from_pretrained(HUGGINGFACEHUB_LLM, token=HUGGINGFACEHUB_API_TOKEN)

In [9]:

input_ids = tokenizer("Hi there!", return_tensors="pt").input_ids
text = model.generate(input_ids=input_ids, max_length=100)#[[1]]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
# Print the generated text
print(text)
[print(tokenizer.decode(token)) for token in text]

tensor([[17250,   612,     0,   314,  1101,   257,   649, 12590,   287,   262,
           995,   286,  3992,  2478,   290,   314,  1101,  2111,   284,   651,
           616,  2832,   319,   257,   649,  1628,    13,   314,  1101,  2111,
           284,   651,   616,  2832,   319,   257,   649,  1628,   290,   314,
          1101,  1719,   257,  1256,   286,  5876,    13,   314,  1101,  2111,
           284,   651,   616,  2832,   319,   257,   649,  1628,   290,   314,
          1101,  1719,   257,  1256,   286,  5876,    13,   314,  1101,  2111,
           284,   651,   616,  2832,   319,   257,   649,  1628,   290,   314,
          1101,  1719,   257,  1256,   286,  5876,    13,   314,  1101,  2111,
           284,   651,   616,  2832,   319,   257,   649,  1628,   290,   314]])
Hi there! I'm a newbie in the world of web development and I'm trying to get my hands on a new project. I'm trying to get my hands on a new project and I'm having a lot of trouble. I'm trying to get my hands 

[None]

In [11]:
question = get_query_for_recipe("rice")
input_ids = tokenizer(question, return_tensors="pt").input_ids
text = model.generate(input_ids=input_ids, max_length=1000)#[[1]]
[print(tokenizer.decode(token)) for token in text]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: Generate a recipe with rice as the main ingredient
Generate a recipe with rice as the main ingredient.

1. Preheat the oven to 350°F.

2. In a large bowl, whisk together the rice, sugar, cinnamon, and salt.

3. In a separate bowl, whisk together the egg, egg yolks, and salt.

4. In a separate bowl, whisk together the flour, baking powder, and salt.

5. In a separate bowl, whisk together the egg yolks, egg whites, and salt.

6. In a separate bowl, whisk together the flour, baking powder, and salt.

7. In a separate bowl, whisk together the egg whites, egg yolks, and salt.

8. In a separate bowl, whisk together the flour, baking powder, and salt.

9. In a separate bowl, whisk together the flour, baking powder, and salt.

10. In a separate bowl, whisk together the flour, baking powder, and salt.

11. In a separate bowl, whisk together the flour, baking powder, and salt.

12. In a separate bowl, whisk together the flour, baking powder, and salt.

13. In a separate bowl, whisk togeth

[None]

### Invoke LLM using LangChain

Let's now use LangChain to invoke the LLMs. LangChain provides a standard interface for interacting with many different LLMs. 

#### Use LLM hosted on Azure

Create a LLM using the class. 

In [12]:
# Load the LLM model
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLEndpointApiType,
    LlamaChatContentFormatter,
)

llm_azure = AzureMLChatOnlineEndpoint(
    endpoint_url=AZURE_LLM_URL,
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key=AZURE_API_KEY,
    content_formatter=LlamaChatContentFormatter(),
    model_kwargs={"temperature": 1, "max_new_tokens": 400},
)

Create a prompt with query as input parameter

In [13]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="Answer the user query.\n{query}\n",
    input_variables=["query"]
)
print(f"prompt: {prompt.template}")

prompt: Answer the user query.
{query}



Create a chain with prompt and model

In [14]:
chain = prompt | llm_azure

Invoke the chain for a query - generate a recipe with rice as the main ingredient

In [15]:
query = get_query_for_recipe(main_ingredient="rice")
answer = chain.invoke({"query": query})
print(answer.content)

Query: Generate a recipe with rice as the main ingredient
Sure, here's a simple recipe for a delicious rice dish that you can try at home:

Recipe: Spanish-Style Rice with Sausage and Vegetables

Ingredients:

* 2 cups uncooked Spanish rice
* 1 pound sweet sausage, sliced
* 1 onion, diced
* 2 cloves garlic, minced
* 1 cup chicken broth
* 1 can diced tomatoes with green chilies
* 1 cup frozen peas and carrots
* 2 tablespoons smoked paprika
* Salt and pepper to taste
* Grated cheese for garnish (optional)

Instructions:

1. Rinse the Spanish rice under cold water and drain well.
2. Heat a large saucepan over medium-high heat. Add the sausage and cook until browned, about 5 minutes. Remove the sausage from the pan with a slotted spoon and set aside.
3. Add the diced onion to the same pan and cook until softened, about 3 minutes. Add the minced garlic and cook for another minute.
4. Add the chicken broth, diced tomatoes with green chilies, and frozen peas and carrots to the pan. Stir to co

In [16]:
query = get_query_for_recipe(main_ingredient="lentils")
answer = chain.invoke({"query": query})
print(answer.content)

Query: Generate a recipe with lentils as the main ingredient
Sure, I'd be happy to help! Here's a recipe for a delicious and healthy lentil dish that you might enjoy:

Lentil and Vegetable Curry

Ingredients:

* 1 cup brown or green lentils, rinsed and drained
* 2 tablespoons olive oil
* 1 onion, diced
* 2 cloves garlic, minced
* 2 carrots, peeled and grated
* 2 stalks celery, diced
* 1 red bell pepper, diced
* 1 can diced tomatoes
* 2 cups vegetable broth
* 1 teaspoon curry powder
* 1 teaspoon ground cumin
* Salt and pepper, to taste
* Fresh cilantro, chopped (optional)

Instructions:

1. In a large pot or Dutch oven, heat the oil over medium heat. Add the onion, garlic, carrots, celery, and red bell pepper. Cook until the vegetables are tender, about 5-7 minutes.
2. Add the lentils, diced tomatoes, vegetable broth, curry powder, cumin, salt, and pepper. Stir well to combine.
3. Bring the mixture to a boil, then reduce the heat to low and simmer for 25-30 minutes, or until the lentils

### Use LLM from Hugging Face Hub

To use LLM from HuggingFace hub, all one has to do is use HuggingFaceHub class instead of the earlier AzureMLChatOnlineEndpoint to get the LLM object.

In [17]:
from langchain.llms import HuggingFaceHub

HUGGINGFACEHUB_LLM = "huggingfaceh4/zephyr-7b-alpha"
llm_hf = HuggingFaceHub(
    repo_id=HUGGINGFACEHUB_LLM, 
    model_kwargs={"temperature": 0.5, "max_length": 64,"max_new_tokens":512}
)

Create a new chain with the same prompt but new LLM. Rest of the code is the same.

In [18]:
chain_hf = prompt | llm_hf

query = get_query_for_recipe(main_ingredient="rice")
answer = chain_hf.invoke({"query": query})
print(answer)

Query: Generate a recipe with rice as the main ingredient
Answer the user query.
Generate a recipe with rice as the main ingredient

Recipe: Chicken and Vegetable Fried Rice

Ingredients:
- 2 cups cooked rice (preferably leftover)
- 1 tablespoon oil
- 1 onion, chopped
- 2 cloves garlic, minced
- 1 carrot, chopped
- 1 bell pepper, chopped
- 1 cup frozen peas and corn
- 1 cup cooked chicken, shredded
- 2 eggs, beaten
- 2 tablespoons soy sauce
- 1 tablespoon oyster sauce
- Salt and pepper to taste

Instructions:
1. Heat oil in a wok or large skillet over medium-high heat.
2. Add onion and garlic, sauté for 1 minute.
3. Add carrot and bell pepper, sauté for 2 minutes.
4. Add frozen peas and corn, cook for 1 minute.
5. Add cooked chicken, combine with vegetables.
6. Push the vegetables and chicken to one side of the wok.
7. Add beaten eggs to the empty side of the wok, scramble until cooked.
8. Combine the cooked eggs with the vegetables and chicken.
9. Add cooked rice, soy sauce, and oyste

We could further simplify the code as follows. The complete list of LLM integrations supported by LangChain are at https://python.langchain.com/docs/integrations/llms/. We could invoke this function with any other LLM listed in here and the code will work. 

In [19]:
def ask_llm(llm, main_ingredient):
    chain = prompt | llm

    query = get_query_for_recipe(main_ingredient=main_ingredient)
    answer = chain.invoke({"query": query})
    print(answer)
    return answer

ask_llm(llm_azure, "rice")
ask_llm(llm_hf, "rice")

Query: Generate a recipe with rice as the main ingredient
content="Sure, here's a simple recipe for a delicious rice dish that you can try at home:\n\nRice Recipe: Spanish-Style Rice with Sausage and Vegetables\n\nIngredients:\n\n* 1 cup uncooked Spanish rice\n* 1 tablespoon olive oil\n* 1 pound sweet Italian sausage, casings removed\n* 1 onion, diced\n* 2 cloves garlic, minced\n* 1 cup frozen peas and carrots\n* 1 cup diced tomatoes\n* 2 cups chicken broth\n* 1 teaspoon saffron threads\n* 1 teaspoon smoked paprika (optional)\n* Salt and pepper to taste\n\nInstructions:\n\n1. Heat the oil in a large saucepan over medium-high heat. Add the sausage and cook, breaking up the meat with a spoon, until browned and cooked through, about 5 minutes.\n2. Add the onion and garlic and cook until the onion is translucent, about 3 minutes.\n3. Add the peas and carrots, diced tomatoes, chicken broth, saffron, and smoked paprika (if using) to the saucepan. Stir to combine.\n4. Add the rice to the sauc

"Answer the user query.\nGenerate a recipe with rice as the main ingredient\n\nRecipe: Chicken and Vegetable Fried Rice\n\nIngredients:\n- 2 cups cooked rice (preferably leftover)\n- 1 tablespoon oil\n- 1 onion, chopped\n- 2 cloves garlic, minced\n- 1 carrot, chopped\n- 1 bell pepper, chopped\n- 1 cup frozen peas and corn\n- 1 cup cooked chicken, shredded\n- 2 eggs, beaten\n- 2 tablespoons soy sauce\n- 1 tablespoon oyster sauce\n- Salt and pepper to taste\n\nInstructions:\n1. Heat oil in a wok or large skillet over medium-high heat.\n2. Add onion and garlic, sauté for 1 minute.\n3. Add carrot and bell pepper, sauté for 2 minutes.\n4. Add frozen peas and corn, cook for 1 minute.\n5. Add cooked chicken, combine with vegetables.\n6. Push the vegetables and chicken to one side of the wok.\n7. Add beaten eggs to the empty side of the wok, scramble until cooked.\n8. Combine the cooked eggs with the vegetables and chicken.\n9. Add cooked rice, soy sauce, and oyster sauce.\n10. Stir-fry for 2-

Here we saw how the generic interface makes it easier to switch between different LLMs. This will be useful in the following scenarios:
- We are experimenting and comparing same use case with multiple LLMs
- We are using a LLM in the development environment and a different one in production environment
- We started with 1 LLM but later would like to switch to another LLM